In [ ]:
import matplotlib.pyplot as plt
import osmnx as ox
import networkx as nx
import sys
import plotly.express as px
import pandas as pd
from pandarallel import pandarallel
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from src.tools import JsonHandler, concatenate_listings_datasets, return_cleaned_col_names, preprocess_text
from src.class_transformers import (
    GeographicTransformer,
    BathroomsTransformer,
    CreateVerificationsTransformer,
    AmenitiesTransformer,
    OfflineLocationFinder,
    PropertyTypeTransformer,
    HostLocationImputer,
    ScrapingDateTransformer
)
from src.function_transformers import (
    fun_tr_id_to_string,
    fun_tr_from_string_to_rate,
    fun_tr_transform_to_datetime,
    fun_tr_remove_dollar_sign,
)
from sklearn.utils import estimator_html_repr
from sklearn import set_config
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob

set_config(transform_output="pandas")

pandarallel.initialize()
pd.options.display.float_format = "{:.0f}".format
handler = JsonHandler()
import pandas as pd
import numpy as np
from feature_engine.datetime import DatetimeSubtraction
from feature_engine.creation import RelativeFeatures
from feature_engine.encoding import OneHotEncoder, CountFrequencyEncoder, OrdinalEncoder
from feature_engine.wrappers import SklearnTransformerWrapper
from feature_engine.imputation import MeanMedianImputer, CategoricalImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer, MaxAbsScaler, PolynomialFeatures, PowerTransformer, RobustScaler
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_squared_error, r2_score, make_scorer, mean_absolute_percentage_error
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import QuantileRegressor
from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import GridSearchCV
import sys
from sklearn.neural_network import MLPRegressor
import keras
from scikeras.wrappers import KerasRegressor

In [ ]:
#df_listings = concatenate_listings_datasets()
#host_locations = handler.import_from_json("data/mappings/host_locations.json")
#remap_baths = handler.import_from_json("data/mappings/baths.json")

In [ ]:
#df_listings.drop(
#    [
#        "neighborhood_overview",
#        "host_about",
#        "host_neighbourhood",
#        "neighbourhood",
#        "neighbourhood_group_cleansed",
#        "calendar_updated",
#        "license",
#        "listing_url",
#        "scrape_id",
#        "last_scraped",
#        "source",
#        "name",
#        "picture_url",
#        "host_url",
#        "host_name",
#        "host_thumbnail_url",
#        "host_picture_url",
#        "minimum_minimum_nights",
#        "maximum_minimum_nights",
#        "minimum_maximum_nights",
#        "maximum_maximum_nights",
#        "minimum_nights_avg_ntm",
#        "maximum_nights_avg_ntm",
#        "has_availability",
#        "availability_30",
#        "availability_60",
#        "availability_90",
#        "availability_365",
#        "calendar_last_scraped",
#        "number_of_reviews_ltm",
#        "number_of_reviews_l30d",
#        "instant_bookable",
#        "calculated_host_listings_count",
#        "calculated_host_listings_count_entire_homes",
#        "calculated_host_listings_count_private_rooms",
#        "calculated_host_listings_count_shared_rooms",
#    ],
#    axis=1,
#    inplace=True
#)

In [ ]:
#df_listings.set_index("id", inplace=True)
#
#df_nas_columns = pd.DataFrame(
#    {
#        "NAs": df_listings.isnull().sum(axis=1),
#        "Columns_with_NAs": df_listings.apply(
#            lambda x: ", ".join(x.index[x.isnull()]), axis=1
#        ),
#    }
#)
#
#more_than_7_missing = df_nas_columns.loc[df_nas_columns["NAs"] > 7, :].index.tolist()
#df_listings.drop(more_than_7_missing, inplace=True)

In [ ]:
#id_feature = ["host_id"]
#rate_feature = ["host_response_rate", "host_acceptance_rate"]
#time_feature = ["host_since", "first_review", "last_review"]
#neighbourhood_feature = ["neighbourhood_cleansed"]
#price_feature = ["price"]
#
## Amenities
#internet_pattern: str = r"\b(wifi|internet|ethernet|fibra|connection)\b"
#self_checkin_pattern: str = r"\b(self checkin|self check-in|self-checkin)\b"
#host_greeting_pattern: str = r"\b(host greeting|host greets you)\b"
#pool_pattern: str = r"\b(pool|pool view|shared pool)\b"
#oven_pattern: str = r"\b(oven)\b"
#microwave_pattern: str = r"\b(microwave|microonde)\b"
#garden_pattern: str = r"\b(garden|park|backyard)\b"
#streaming_pattern: str = r"\b(netflix|amazon|disney+|chromecast|apple tv|hbo|hbo max)\b"
#gym_pattern: str = r"\b(exercise|gym|fitness|private gym in building|shared gym|gym nearby|workout bench)\b"
#elevator_pattern: str = r"\b(elevator)\b"
#heating_pattern: str = r"\b(heating)\b"
#ac_pattern: str = r"\b(central air conditioning|ac|air conditioning)\b"
#safe_pattern: str = r"\b(safe|locker|lock|security|guard)\b"
#workspace_pattern: str = r"\b(workspace|work)\b"
#freezer_pattern: str = r"\b(freezer|refrigerator)\b"
#aid_pattern: str = r"\b(first aid kit|aid)\b"
#dishwasher_pattern: str = r"\b(dishwasher)\b"
#long_term_stays_pattern: str = r"\b(long term stays)\b"
#pets_pattern: str = r"\b(pets allowed)\b"
#bathtube_pattern: str = r"\b(bathtube)\b"
#bbq_grill_pattern: str = r"\b(bbq grill|grill|barbeque|barbeque utensils)\b"
#lake_bay_pattern: str = r"\b(lake view|bay view|harbor view|beach view)\b"
#
#set_amenities_remapper = [
#    (internet_pattern, "internet"),
#    (self_checkin_pattern, "self-checkin"),
#    (host_greeting_pattern, "host-greeting"),
#    (pool_pattern, "pool"),
#    (oven_pattern, "oven"),
#    (microwave_pattern, "microwave"),
#    (garden_pattern, "garden"),
#    (streaming_pattern, "streaming"),
#    (gym_pattern, "gym"),
#    (elevator_pattern, "elevator"),
#    (heating_pattern, "heating"),
#    (ac_pattern, "air-conditioning"),
#    (workspace_pattern, "workspace"),
#    (freezer_pattern, "freezer"),
#    (aid_pattern, "first-aid-kit"),
#    (dishwasher_pattern, "dishwasher"),
#    (long_term_stays_pattern, "long-term-stays"),
#    (pets_pattern, "pets-allowed"),
#    (bathtube_pattern, "bathtube"),
#    (bbq_grill_pattern, "bbq-grill"),
#    (lake_bay_pattern, "lake-bay-view")
#]
#
## Property type
#entire_property_pattern = r"\b(entire|tiny home)\b"
#private_room_pattern = r"\b(private room|room in serviced apartment|room in bed and breakfast|room in hotel|room in #resort)\b"
#shared_room_pattern = r"\b(shared room|shared)\b"
#other_room_pattern = r"\b(entire|tiny home|private room|room in serviced apartment|room in bed and breakfast|room in #hotel|room in resort|shared room|shared)\b"
#
#set_property_type_remapper = [
#    (entire_property_pattern, "entire_property"),
#    (private_room_pattern, "private_room"),
#    (shared_room_pattern, "shared_room"),
#    (other_room_pattern, "other"),
#]
#
#id_pipeline = Pipeline(steps=[("From ID to string", fun_tr_id_to_string)], verbose=True)
#
#rates_pipeline = Pipeline(
#    steps=[("Transform response rate", fun_tr_from_string_to_rate)], verbose=True
#)
#
#timestamp_pipeline = Pipeline(
#    steps=[("Transform to timestamp", fun_tr_transform_to_datetime)], verbose=True
#)
#
#price_pipeline = Pipeline(
#    steps=[("Trim price feature", fun_tr_remove_dollar_sign)], verbose=True
#)
#
## Apply to all dataset (feature engineering using other features)
#feature_creation_pipeline = Pipeline(
#    steps=[
#        ("Listing Locations", OfflineLocationFinder()),
#        ("Host Locations imputer", HostLocationImputer()),
#        (
#            "Host location",
#            GeographicTransformer(column="host_location", locations=host_locations),
#        ),
#        ("Host verifications", CreateVerificationsTransformer()),
#        ("Bathrooms", BathroomsTransformer(remap_baths)),
#        (
#            "Amenities",
#            AmenitiesTransformer(df=df_listings, remapper=set_amenities_remapper),
#        ),
#        (
#            "Property type",
#            PropertyTypeTransformer(
#                df=df_listings, remapper=set_property_type_remapper
#            ),
#        ),
#    ],
#    verbose=True,
#)
#print("Executing Feature Creation Pipeline...")
#df_listings = feature_creation_pipeline.fit_transform(df_listings)
#print("Feature Creation Pipeline completed!")


In [ ]:
#print("Executing preprocessing on features...")
#feature_preprocessor = ColumnTransformer(
#    remainder="passthrough",
#    n_jobs=-1,
#    force_int_remainder_cols=False,
#    transformers=[
#        ("Id", id_pipeline, id_feature),
#        ("Rates", rates_pipeline, rate_feature),
#        ("Price", price_pipeline, price_feature),
#        ("Timestamp", timestamp_pipeline, time_feature),
#    ],
#    verbose=True,
#)
#
#cleaned_df = feature_preprocessor.fit_transform(df_listings)
#print("Preprocessing on features completed!")
#cleaned_df.columns = return_cleaned_col_names(cleaned_df.columns)
#print("Cleaned feature names retrieved")
#
#del df_listings

In [ ]:
#pd.to_pickle(cleaned_df, "data/city_data/step_by_step.pkl")

In [ ]:
df = pd.read_pickle("data/city_data/step_by_step.pkl")

## Resizing of dataset

In [ ]:
df.shape

## Feature separation

In [ ]:
numerical_features = [
    'host_response_rate',
    'host_acceptance_rate',
    'price',
    'host_location',
    'host_listings_count',
    'host_total_listings_count',
    'latitude',
    'longitude',
    'accommodates',
    'bathrooms',
    'bedrooms',
    'beds',
    'minimum_nights',
    'maximum_nights',
    'number_of_reviews',
    'review_scores_rating',
    'review_scores_accuracy',
    'review_scores_cleanliness',
    'review_scores_checkin',
    'review_scores_communication',
    'review_scores_location',
    'review_scores_value',
    'reviews_per_month',
    'listing_city_pop',
]

In [ ]:
binary_features = [
    'host_has_profile_pic',
    'host_identity_verified',
    "host_is_superhost",
    'email_verification',
    'phone_verification',
    'work_email_verification',
    'amenities_internet',
    'amenities_self-checkin',
    'amenities_host-greeting',
    'amenities_pool',
    'amenities_oven',
    'amenities_microwave',
    'amenities_garden',
    'amenities_streaming',
    'amenities_gym',
    'amenities_elevator',
    'amenities_heating',
    'amenities_air-conditioning',
    'amenities_workspace',
    'amenities_freezer',
    'amenities_first-aid-kit',
    'amenities_dishwasher',
    'amenities_long-term-stays',
    'amenities_pets-allowed',
    'amenities_bathtube',
    'amenities_bbq-grill',
    'amenities_lake-bay-view'
]

In [ ]:
other_features = list(set(df.columns.tolist()) - set(numerical_features) - set(binary_features))

## Correlation between features

In [ ]:
df_corr = df.copy()
corr_full = df_corr[numerical_features].corr()
colors = ['green' if val < 0.7 else 'red' for val in corr_full.values.flatten()]
mask = np.where(corr_full < 0.7, 0, 1)

fig = px.imshow(corr_full,
                text_auto=True,
                aspect="auto",
                width=900,
                height=900
                )

fig.show()

In [ ]:
fig = px.imshow(corr_full,
                text_auto=True,
                aspect="auto",
                width=900,
                height=900,
                color_continuous_scale=["green", "red"])

fig.update_traces(z=mask, colorscale=[[0, "green"], [1, "red"]])

fig.show()

### Create some features

In [ ]:
df["beds_for_bedroom"] = np.where(df["bedrooms"] != 0, df["beds"] / df["bedrooms"], 0)

### Drop highly correlates

In [ ]:
numerical_features = [
    "beds_for_bedroom",
    'host_response_rate',
    'host_acceptance_rate',
    'price',
    'host_location',
    'host_listings_count',
    'latitude',
    'longitude',
    'accommodates',
    'bathrooms',
    'minimum_nights',
    'maximum_nights',
    'number_of_reviews',
    'review_scores_rating',
    'review_scores_checkin',
    'review_scores_location',
    'review_scores_value',
    'reviews_per_month',
    'listing_city_pop',
]

In [ ]:
df_corr = df.copy()
corr_full = df_corr[numerical_features].corr()

fig = px.imshow(corr_full,
                text_auto=True,
                aspect="auto",
                width=900,
                height=900
                )

fig.show()

In [ ]:
other_features = list(set(df.columns.tolist()) - set(numerical_features) - set(binary_features))

### Exploration

*Data exploration of features. Firstly the binary ones with respect to price*.

*Then engineer the other features as needed and observe their relationship with price*.

In [ ]:
#for var in binary_features:
#    fig = px.histogram(df,
#                       x = f"{var}",
#                       color="df_city_location",
#                       barmode='group'
#                       )
#    fig.show()

## Managing Other Features

In [ ]:
other_features

In [ ]:
df["review_scores_communication"].unique()

In [ ]:
ohe_features = [
    "property_type",
    "df_city_location",
    "room_type",
    "host_response_time",
    "bathrooms_text"
]

In [ ]:
other_features = list(set(other_features) - set(ohe_features))

## Prediction

In [ ]:
testing = df.drop(other_features, axis=1)

In [ ]:
testing.replace({"f": 0, "t": 1}, inplace=True)

In [ ]:
testing.reset_index(inplace=True)
testing.drop(["id"], inplace=True, axis=1)

In [ ]:
testing.dropna(inplace=True)

In [ ]:
train_set, test_set = train_test_split(testing, test_size=0.2, random_state=42)

In [ ]:
train_set = train_set.loc[train_set["price"]<500, :]

In [ ]:
fig = px.histogram(train_set,
                   x = "price")
fig.show()


In [ ]:
X_train = train_set.drop(["price"], axis=1)
X_test = test_set.drop(["price"], axis=1)

y_train = train_set["price"]
y_test = test_set["price"]

In [ ]:
def create_model():
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(units=64, activation='relu', input_shape=(input_dim,)))  # Change input_dim to your input feature size
    model.add(keras.layers.Dense(units=32, activation='relu'))  # Hidden layer
    model.add(keras.layers.Dense(units=1, activation='linear'))  # Output layer
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [ ]:
model = KerasRegressor(
    model=create_model,
    epochs=100,
    batch_size=500,
    verbose=1
)

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', model)
])

In [ ]:
wizard_pipe = Pipeline(
    steps=[
        (
            "OHE_imputation",
            CategoricalImputer(
                imputation_method="frequent",
                variables=ohe_features,
                return_object=True,
                ignore_format=False,
            ),
        ),
        (
            "OHE_encoding",
            OneHotEncoder(
                top_categories=None,
                drop_last=True,
                drop_last_binary=True,
                ignore_format=False,
                variables=ohe_features,
            ),
        ),
        (
            "Standardize",
            StandardScaler(),
        ),
        #(
        #    "Model",
        #    RandomForestRegressor(
        #        n_estimators=100,
        #        criterion="absolute_error",
        #        bootstrap=True,
        #        max_samples=0.8,
        #        oob_score=True,
        #        n_jobs=-1,
        #        random_state=874631,
        #        verbose=True
        #    ),
        #),
        #(
        #   "KNeighborsRegressor",
        #   KNeighborsRegressor(
        #       n_neighbors=5,
        #       weights="uniform",
        #       algorithm="auto",
        #       n_jobs=-1,
        #   )
        #),
        #(
        #    "GridSearchCV",
        #    GridSearchCV(
        #        TransformedTargetRegressor(
        #            regressor=MLPRegressor(),
        #            transformer=PowerTransformer(),
        #        ),
        #        param_grid=param_grid,
        #        refit="r2_score",
        #        scoring=scoring,
        #        #n_jobs=-1,
        #        pre_dispatch=4, # avoid jobs explosion
        #        cv=None,        # use default 5 fold cross val,
        #        verbose=4,
        #        return_train_score=True
        #    ),
        #),
        #(
        #    "MLPRegressor",
        #    MLPRegressor(hidden_layer_sizes=(100, 100,),
        #                 activation="logistic",
        #                 solver="adam",
        #                 alpha=0.001,
        #                 batch_size="auto",
        #                 learning_rate="constant",
        #                 learning_rate_init=0.001,
        #                 max_iter=200,
        #                 shuffle=True,
        #                 random_state=874631,
        #                 tol=1e-4,
        #                 verbose=True,
        #                 warm_start=False,
        #                 momentum=0.9,
        #                 early_stopping=True,
        #                 validation_fraction=0.3,
        #                 n_iter_no_change=50
        #    )
        #),
        #(
        #    "SupportVectorRegression",
        #    SVR(
        #        kernel="rbf",
        #        gamma="auto",
        #        tol=1e-3,
        #        epsilon=0.1,
        #        verbose=True
        #    ),
        #),
        (
            "Model",
            TransformedTargetRegressor(
                regressor=MLPRegressor(),
                transformer=PowerTransformer(),
            ),
        )
    ],
    verbose=True
)

In [ ]:
#param_grid = {
#    'Standardize': [StandardScaler(), MinMaxScaler(), Normalizer(), MaxAbsScaler()],
#    'Model__hidden_layer_sizes': [(100, 100, 100)],# [(10, 10), (10, 10, 10), (100, 100), (100, 100, 100)],  #, (100, 50), (150, 100, 50)],
#    'Model__activation': ['logistic'],
#    'Model__solver': ['adam'],
#    'Model__alpha': [0.01],
#    'Model__learning_rate': ['constant'], #, 'invscaling', 'adaptive'],
#    'Model__learning_rate_init': [0.001], #, 0.01, 0.1],
#    'Model__random_state':[874631],
#    'Model__tol':[1e-4],
#    'Model__verbose':[True],
#    #'Model__momentum': [0.8, 0.85, 0.9, 0.95, 0.99],
#    'Model__n_iter_no_change': [50],
#    'Model__max_iter': [300],
#    'Model__early_stopping': [True],
#    #'Model__validation_fraction': [0.1, 0.2, 0.3],
#    #'transformer__method': ['yeo-johnson'],
#    #'transformer__standardize': [True],
#    #'transformer__copy': [False]
#}
#
#scoring = {
#    'mean_absolute_error': make_scorer(mean_absolute_error, greater_is_better=False),
#    'mean_squared_error': make_scorer(mean_squared_error, greater_is_better=False),
#    'r2_score': make_scorer(r2_score, greater_is_better=True),
#}

In [ ]:
param_grid = {
    'Standardize': [StandardScaler(), MinMaxScaler(), Normalizer(), MaxAbsScaler()],
    'Model__regressor__hidden_layer_sizes': [(100, 100, 100)],# [(10, 10), (10, 10, 10), (100, 100), (100, 100, 100)],  #, (100, 50), (150, 100, 50)],
    'Model__regressor__activation': ['logistic'],
    'Model__regressor__solver': ['adam'],
    'Model__regressor__alpha': [0.01],
    'Model__regressor__learning_rate': ['constant'], #, 'invscaling', 'adaptive'],
    'Model__regressor__learning_rate_init': [0.001], #, 0.01, 0.1],
    'Model__regressor__random_state':[874631],
    'Model__regressor__tol':[1e-4],
    'Model__regressor__verbose':[True],
    'Model__regressor__momentum': [0.8, 0.85, 0.9, 0.95, 0.99],
    'Model__regressor__n_iter_no_change': [50],
    'Model__regressor__max_iter': [300],
    'Model__regressor__early_stopping': [True],
    'Model__regressor__validation_fraction': [0.1, 0.2, 0.3],
    'Model__transformer__method': ['yeo-johnson'],
    'Model__transformer__standardize': [True],
    'Model__transformer__copy': [False]
}

scoring = {
    'mean_absolute_error': make_scorer(mean_absolute_error, greater_is_better=False),
    'mean_squared_error': make_scorer(mean_squared_error, greater_is_better=False),
    'r2_score': make_scorer(r2_score, greater_is_better=True),
}

In [ ]:
grid_pipeline = GridSearchCV(
    wizard_pipe,
    param_grid=param_grid,
    refit="mean_absolute_error",
    scoring=scoring,
    n_jobs=-1,
    pre_dispatch=4, # avoid jobs explosion
    cv=2,        # use default 5 fold cross val,
    verbose=4,
    return_train_score=True 
    )

grid_pipeline.fit(X_train, y_train)

In [ ]:
grid_pipeline.best_params_

In [ ]:
best_pipe = grid_pipeline.best_estimator_
print(best_pipe)

In [ ]:
wizard_pipe = Pipeline(
    steps=[
        (
            "OHE_imputation",
            CategoricalImputer(
                imputation_method="frequent",
                variables=ohe_features,
                return_object=True,
                ignore_format=False,
            ),
        ),
        (
            "OHE_encoding",
            OneHotEncoder(
                top_categories=None,
                drop_last=True,
                drop_last_binary=True,
                ignore_format=False,
                variables=ohe_features,
            ),
        ),
        (
            "Standardize",
            StandardScaler(),
        ),
        (
            "Model",
            (grid_pipeline.best_params)
        )
    ],
    verbose=True
)

In [ ]:
best_pipe.fit(X_train, y_train)

In [ ]:
pred = best_pipe.predict(X_test)
print(
    f"\nExplained variance score is {explained_variance_score(y_true=y_test, y_pred=pred)}",
    f"\nMean Absolute Error is {mean_absolute_error(y_true=y_test, y_pred=pred)}",
    f"\nMean Absolute Percentage error is {round(100*mean_absolute_percentage_error(y_true=y_test, y_pred=pred), 2)}%",
    f"\nMean Squared Error is {mean_squared_error(y_true=y_test, y_pred=pred)}",
    f"\nR^2 Error is {r2_score(y_true=y_test, y_pred=pred)}",
)

In [ ]:
results = pd.DataFrame(data={"Pred": pred, "y_test": y_test})

In [ ]:
results["Difference"] = abs(results["Pred"]-results["y_test"])

In [ ]:
sum(results["Difference"]>10)/results.shape[0]